# Day 1: Introduction to AmberTools & Small Molecule Parameterization

This notebook introduces **AmberTools**, its sub-programs, and how they are used for
building and parameterizing small organic molecules for molecular dynamics workflows.

## What is AmberTools?
AmberTools is an open‑source suite used for:
- Preparing molecules for MD simulations
- Generating force‑field parameters (GAFF, GAFF2)
- File manipulation & conversions
- Geometry optimization and charge assignment

## Key AmberTools programs covered today

**1. antechamber**
- Generates atom types (GAFF/GAFF2)
- Computes partial charges (AM1-BCC, RESP)
- Produces parameter files (`.mol2`, `.prepi`, `.frcmod`)

**2. parmchk2**
- Checks for missing parameters
- Generates a `.frcmod` file containing supplemental parameters

**3. tleap**
- Assembles the full system (molecule, solvent, ions)
- Writes AMBER `prmtop` and `inpcrd`

We will also visualize the molecule using **nglview**.

# Small molecule parameterisation
## Step 0: Initialization

This step sets up the environment for the tutorial. It installs required Python packages, creates necessary directories, and prints the current working directory. 

**Important:** Run this step only once when starting the notebook or after restarting the kernel.

In [10]:
# --- Lightweight "reset" for clean re-run ---
from IPython.display import clear_output
import importlib, os, sys

clear_output(wait=True)
importlib.invalidate_caches()

# Install (safe even if already installed)
!{sys.executable} -m pip install nglview parmed > /dev/null

import nglview as nv
import parmed as pmd

# 1. Set fixed project root ONE time
if "PROJECT_ROOT" not in os.environ:
    os.environ["PROJECT_ROOT"] = os.path.abspath(os.getcwd())

PROJECT_ROOT = os.environ["PROJECT_ROOT"]
print("Project root:", PROJECT_ROOT)

# 2. Create subdirectories safely
sm_dir = os.path.join(PROJECT_ROOT, "small_molecule")
mono_dir = os.path.join(PROJECT_ROOT, "monomer")

os.makedirs(sm_dir, exist_ok=True)
os.makedirs(mono_dir, exist_ok=True)

print("small_molecule path:", sm_dir)
print("monomer path:", mono_dir)

Project root: /Users/harish/Documents/Work_folder/MetaChem-workshops/polymermd-workshop/Day1-Ambertools-solved
small_molecule path: /Users/harish/Documents/Work_folder/MetaChem-workshops/polymermd-workshop/Day1-Ambertools-solved/small_molecule
monomer path: /Users/harish/Documents/Work_folder/MetaChem-workshops/polymermd-workshop/Day1-Ambertools-solved/monomer


## Step 1: Load a small molecule (ethanol example)

In this step, we will:
1. Use OpenBabel to convert a SMILES string into a 3D molecular structure in MOL2 format.
2. Save the resulting file in the `small_molecule` directory.

Ensure OpenBabel is installed in your environment before running this step.

In [2]:
# Set up the working directory
small_molecule_dir = f"{PROJECT_ROOT}/small_molecule"
os.makedirs(small_molecule_dir, exist_ok=True)
os.chdir(small_molecule_dir)

# Define the molecule
smiles = "CCCO"  # Ethanol
mol_name = "propanol"  # Define molecule name

# Convert SMILES to 3D MOL2 using OpenBabel
command = f"obabel -:{smiles} -O {mol_name}.mol2 --gen3d"
print(f"Executing: {command}")
os.system(command)

Executing: obabel -:CCCO -O propanol.mol2 --gen3d


1 molecule converted


0

## Visualize the molecule

This step uses `nglview` to visualize the 3D structure of the molecule generated in the previous step.

In [3]:
# Load and visualize the molecule
mol_name = "propanol"  # Ensure mol_name is defined
view = nv.show_file(f"{small_molecule_dir}/{mol_name}.mol2")
view

NGLWidget()

## Step 2: Run `antechamber` to create GAFF atom types and AM1-BCC charges

This step uses the `antechamber` program to:
1. Assign GAFF atom types to the molecule.
2. Compute AM1-BCC partial charges.
3. Generate a new MOL2 file with the updated parameters.

Ensure `antechamber` is installed and available in your environment.

In [4]:
# Run antechamber to generate GAFF atom types and AM1-BCC charges
mol_name = "propanol"  # Ensure mol_name is defined
command = f"antechamber -i {mol_name}.mol2 -fi mol2 -o {mol_name}_gaff.mol2 -fo mol2 -at gaff2 -c bcc -s 2 -pf y"
print(f"Executing: {command}")
os.system(command)

# Display the output of the sqm calculation
os.system("tail sqm.out")

Executing: antechamber -i propanol.mol2 -fi mol2 -o propanol_gaff.mol2 -fo mol2 -at gaff2 -c bcc -s 2 -pf y
Info: acdoctor mode is on: check and diagnose problems in the input file.
Info: The atom type is set to gaff2; the options available to the -at flag are
      gaff, gaff2, amber, bcc, abcg2, and sybyl.

-- Check Format for mol2 File --
   Status: pass
Info: Finished reading file (propanol.mol2); atoms read (12), bonds read (11).
Info: Determining atomic numbers from atomic symbols which are case sensitive.
Running: /Users/harish/miniconda3/envs/polymer_md/bin/bondtype -j full -i ANTECHAMBER_BOND_TYPE.AC0 -o ANTECHAMBER_BOND_TYPE.AC -f ac

Running: /Users/harish/miniconda3/envs/polymer_md/bin/atomtype -i ANTECHAMBER_AC.AC0 -o ANTECHAMBER_AC.AC -p gaff2

Running: /Users/harish/miniconda3/envs/polymer_md/bin/sqm -O -i sqm.in -o sqm.out

Running: /Users/harish/miniconda3/envs/polymer_md/bin/am1bcc -i ANTECHAMBER_AM1BCC_PRE.AC -o ANTECHAMBER_AM1BCC.AC -f ac -t bcc -p /Users/harish/min

0

## Step 3: Generate supplemental parameters using `parmchk2`

This step uses the `parmchk2` program to:
1. Check for missing force-field parameters.
2. Generate a `.frcmod` file containing supplemental parameters.

Ensure `parmchk2` is installed and available in your environment.

In [5]:
# Run parmchk2 to generate the frcmod file
mol_name = "propanol"  # Ensure mol_name is defined
command = f"parmchk2 -i {mol_name}_gaff.mol2 -f mol2 -o {mol_name}.frcmod"
print(f"Executing: {command}")
os.system(command)

# Display the contents of the frcmod file
os.system(f"cat {mol_name}.frcmod")

Executing: parmchk2 -i propanol_gaff.mol2 -f mol2 -o propanol.frcmod
Remark line goes here
MASS

BOND

ANGLE

DIHE

IMPROPER

NONBON





0

## Step 5: Build topology & coordinates with `tleap`

This step creates a topology and coordinate file for the molecule using `tleap`. It generates the following files:
- `prmtop`: Topology file
- `inpcrd`: Coordinate file
- `pdb`: Optional PDB file for visualization

Ensure `tleap` is installed and available in your environment.

In [6]:
# Create tleap input file
mol_name = "propanol"  # Ensure mol_name is defined
tleap_input = f"""
source leaprc.gaff2
source leaprc.water.tip3p

# Load molecule (from antechamber-generated files)
ETH = loadmol2 {mol_name}_gaff.mol2
loadamberparams {mol_name}.frcmod

# Build topology and coordinate files
saveamberparm ETH {mol_name}.prmtop {mol_name}.inpcrd

# Optional: PDB for visualization
savepdb ETH {mol_name}.pdb
"""

with open("tleap.in", "w") as f:
    f.write(tleap_input)

# Run tleap and ensure it exits after execution
command = "tleap -f tleap.in << 'EOF'\nquit\nEOF"
print(f"Executing: {command}")
os.system(command)

Executing: tleap -f tleap.in << 'EOF'
quit
EOF
-I: Adding /Users/harish/miniconda3/envs/polymer_md/dat/leap/prep to search path.
-I: Adding /Users/harish/miniconda3/envs/polymer_md/dat/leap/lib to search path.
-I: Adding /Users/harish/miniconda3/envs/polymer_md/dat/leap/parm to search path.
-I: Adding /Users/harish/miniconda3/envs/polymer_md/dat/leap/cmd to search path.
-f: Source tleap.in.

Welcome to LEaP!
(no leaprc in search path)
Sourcing: ./tleap.in
----- Source: /Users/harish/miniconda3/envs/polymer_md/dat/leap/cmd/leaprc.gaff2
----- Source of /Users/harish/miniconda3/envs/polymer_md/dat/leap/cmd/leaprc.gaff2 done
Log file: ./leap.log
Loading parameters: /Users/harish/miniconda3/envs/polymer_md/dat/leap/parm/gaff2.dat
Reading title:
AMBER General Force Field for organic molecules (Version 2.2.20, March 2021)
----- Source: /Users/harish/miniconda3/envs/polymer_md/dat/leap/cmd/leaprc.water.tip3p
----- Source of /Users/harish/miniconda3/envs/polymer_md/dat/leap/cmd/leaprc.water.tip


*** Error: tl_getline(): not interactive, use stdio.


256

## Step 6: Convert to GROMACS formats

This step converts the AMBER topology and coordinate files into GROMACS-compatible formats:
- `.gro`: GROMACS structure file
- `.top`: GROMACS topology file

In [7]:
# Load AMBER topology & coordinates
mol_name = "propanol"  # Ensure mol_name is defined
amber = pmd.load_file(f"{mol_name}.prmtop", f"{mol_name}.inpcrd")

# Save GROMACS structure (.gro)
amber.save(f"{mol_name}.gro", overwrite=True)

# Save GROMACS topology (.top)
amber.save(f"topol.top", format="gromacs", overwrite=True)

print("Conversion completed: molecule.gro + topol.top")

Conversion completed: molecule.gro + topol.top


# Monomer parameterisation

## Step 1: Prepare the monomer from SMILES

In this step, we will:
1. Use OpenBabel to convert a SMILES string for a polymer into a 3D molecular structure in MOL2 format.
2. Visualize the resulting structure.

In [8]:
# Set up the working directory
monomer_dir = f"{PROJECT_ROOT}/monomer"
os.makedirs(monomer_dir, exist_ok=True)
os.chdir(monomer_dir)

# Define the polymer
smiles = "CCOCCOCCOCCOCCOCC"  # Polyethylene oxide (PEO)
mol_name = "PEO"  # Define molecule name

# Convert SMILES to 3D MOL2 using OpenBabel
command = f"obabel -:{smiles} -O {mol_name}.mol2 --gen3d"
print(f"Executing: {command}")
os.system(command)

# Visualize the molecule
view = nv.show_file(f"{mol_name}.mol2")
view.add_label(radius=2, color='black', label_type='atomindex')
view

Executing: obabel -:CCOCCOCCOCCOCCOCC -O PEO.mol2 --gen3d


1 molecule converted


NGLWidget()

## Step 2: Run antechamber to create GAFF atom types and AM1-BCC charges

This step uses the `antechamber` program to parameterize the polymer.

In [9]:
# Run antechamber for the polymer
mol_name = "PEO"  # Ensure mol_name is defined
command = f"antechamber -i {mol_name}.mol2 -fi mol2 -o {mol_name}_gaff.ac -fo ac -at gaff2 -an y -c bcc -nc 0 -rn PEO -s 2 -pf y"
print(f"Executing: {command}")
os.system(command)

# Display the output of the sqm calculation
os.system("tail sqm.out")

Executing: antechamber -i PEO.mol2 -fi mol2 -o PEO_gaff.ac -fo ac -at gaff2 -an y -c bcc -nc 0 -rn PEO -s 2 -pf y
Info: acdoctor mode is on: check and diagnose problems in the input file.
Info: The atom type is set to gaff2; the options available to the -at flag are
      gaff, gaff2, amber, bcc, abcg2, and sybyl.

-- Check Format for mol2 File --
   Status: pass
Info: Finished reading file (PEO.mol2); atoms read (43), bonds read (42).
Info: Determining atomic numbers from atomic symbols which are case sensitive.
Running: /Users/harish/miniconda3/envs/polymer_md/bin/bondtype -j full -i ANTECHAMBER_BOND_TYPE.AC0 -o ANTECHAMBER_BOND_TYPE.AC -f ac

Running: /Users/harish/miniconda3/envs/polymer_md/bin/atomtype -i ANTECHAMBER_AC.AC0 -o ANTECHAMBER_AC.AC -p gaff2

Running: /Users/harish/miniconda3/envs/polymer_md/bin/sqm -O -i sqm.in -o sqm.out

Running: /Users/harish/miniconda3/envs/polymer_md/bin/am1bcc -i ANTECHAMBER_AM1BCC_PRE.AC -o ANTECHAMBER_AM1BCC.AC -f ac -t bcc -p /Users/harish/mi

0